In [2]:
%matplotlib widget
import numpy as np
import math
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import random

import matplotlib.pyplot as plt

import re

In [3]:
plik = open("../100k.txt")

slowka = [slowo.lower().replace("\n", "") for index,slowo in enumerate(plik)]
slowa = []

print(len(slowka))

while len(slowa) < 100:
    los = random.randint(0, len(slowka) - 1)
    if slowka[los] not in slowa:
        slowa.append(slowka[los])

print(len(slowa))
print(slowa[0:200])

slowa = slowa[0:2000]

100000
100
['502945220', 'mazur5', 'fno47wnf', 'plujek', 'wsmbydgoszcz', 'wanda12', '18051974', 'blazenek', 'folwark', 'wntkp49', 'h0bb1t', '280470', 'ika1229', 'lidia1970', 'sabra23', 'gadzina', 'ppspike', 'kolec4', 'pawel6', 'zuza22', 'twistor', 'gapa', 'st2309', 'anita2009', 'fender4', 'pikulgks', 'smeagol', 'halina61', 'piksel', 'fargo6', 'raba', 'kasiakasia', 'kacper21', 'becha1', 'asica1972', 'rybaryba', 'aga1981', 'kjuik', 'kierasek', 'poter', 'ddsa12', 'panjanpan', 'monika1', 'queen05', 'nosferatum', 'marlon', 'lukis1', 'lysy1975', 'wnuczek', '100gram', 'pawel1979', 'kurt1970', 'art1011', '2102jozef', 'cewak1111', 'fenix67', 'lama641', 'okmn1bbb', 'anser80', 'orbit1', 'sirius', 'fikimiki', 'almaberta', 'klipo1', '892589', 'lewmar', 'mike99', 'yachtdragon', 'player', 'badura', 'opkny6', '26krisek', 'honcia', 'hiper5', 'pankracy', 'jacky67', 'chujnaryj', 'kambr66', 'dupersznyt', 'lisek1959', 'polimer1', 'pablosan1', 'bacman', 'sandiego', '0101ola', 'pukpuk1', 'mentel', 'warny76',

In [4]:
def find_unique(wektor):
    return np.array(sorted(list({letter for word in wektor for letter in word})))

In [5]:
def make_one_hot_vectors(wektor):
    return {wektor[cnt]:[1 if ii == cnt else 0 for ii in range(len(wektor))] for cnt in range(len(wektor))}

In [6]:
def make_word_vector(word, onehotvectors):
    return np.array([onehotvectors[letter] for letter in word])

In [7]:
def find_longest_word(words):
    longest = 0
    
    for word in words:
        if len(word) > longest:
            longest = len(word)
            
    return longest

find_longest_word(slowa)

12

In [8]:
def get_inputs_length(words, onehotvectors):
    return find_longest_word(words)*len(onehotvectors[list(onehotvectors.keys())[0]])

insy = get_inputs_length(slowa, make_one_hot_vectors(find_unique(slowa)))
print(insy)

420


In [9]:
def get_inputs_by_characters_count(ch_count, onehotvectors):
    return ch_count*len(onehotvectors[list(onehotvectors.keys())[0]])

#insy = get_inputs_by_characters_count(15, make_one_hot_vectors(find_unique(slowa)))
#print(insy)

In [10]:
def build_slowa(words, onehotvectors, wielkoscwejscia):
    wielkosc_one_hota = len(onehotvectors[list(onehotvectors.keys())[0]])
    
    zwrot = []
    
    for slowo in words:
        builded = np.array(make_word_vector(slowo, onehotvectors))
        builded = builded.reshape(-1)
        braki = wielkoscwejscia - len(builded)
        
        wektor = np.concatenate((builded,np.array([0 for __ in range(braki)])), axis=0)
        zwrot.append(wektor)
    return np.array(zwrot)
    
#build_slowa(words, make_one_hot_vectors(find_unique(words)), 150)

In [31]:
class Generator(nn.Module):
    def __init__(self, ins, outs):
        super(Generator, self).__init__()
        
        self.fc_in = nn.Linear(in_features=ins, out_features=2000)
        self.fc_h1 = nn.Linear(in_features=2000, out_features=2000)
        self.fc_h2 = nn.Linear(in_features=2000, out_features=3000)
        self.fc_h3 = nn.Linear(in_features=3000, out_features=4000)
        self.fc_h4 = nn.Linear(in_features=4000, out_features=2000)
        self.fc_h5 = nn.Linear(in_features=2000, out_features=500)
        self.fc_h6 = nn.Linear(in_features=500, out_features=300)
        self.fc_h7 = nn.Linear(in_features=300, out_features=300)
        self.fc_out = nn.Linear(in_features=300, out_features=outs)
        
        self.activation = nn.ReLU()
        self.activation2 = nn.Sigmoid()
        
    def forward(self, x):
        x = self.fc_in(x)
        # x = self.activation(x)
        
        x = self.fc_h1(x)
        # x = self.activation(x)
        
        x = self.fc_h2(x)
        # x = self.activation(x)
        
        x = self.fc_h3(x)
        # x = self.activation(x)
        
        x = self.fc_h4(x)
        # x = self.activation(x)
        
        x = self.fc_h5(x)
        # x = self.activation(x)
        
        x = self.fc_h6(x)
        # x = self.activation(x)
        
        x = self.fc_h7(x)
        x = self.activation2(x)
        
        x = self.fc_out(x)
        #x = self.activation2(x)
        
        return x

In [32]:
class Classifier(nn.Module):
    def __init__(self, ins, outs):
        super(Classifier, self).__init__()
        
        self.fc_in = nn.Linear(in_features=ins, out_features=1000)
        self.fc_h1 = nn.Linear(in_features=1000, out_features=1000)
        self.fc_h2 = nn.Linear(in_features=1000, out_features=1000)
        self.fc_h3 = nn.Linear(in_features=1000, out_features=1000)
        self.fc_out = nn.Linear(in_features=1000, out_features=outs)
        
        self.activation = nn.ReLU()
        self.activation2 = nn.Sigmoid()
        
    def forward(self, x):
        inp = x
        
        x = self.fc_in(x)
        x = self.activation(x)
        
        x = self.fc_h1(x)
        x = self.activation(x)
        
        x = self.fc_h2(x)
        x = self.activation(x)
        
        x = self.fc_h3(x)
        x = self.activation(x)
        
        x = self.fc_out(x)
        x = self.activation2(x)
        
        return x

In [33]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.xavier_normal(m.weight.data, 1.0, 0.002)
        #nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [47]:
manualSeed = 1001
random.seed(manualSeed)
torch.manual_seed(manualSeed)

In [48]:
TensorRealOuts = torch.full((len(slowa), insy), 1, requires_grad=False).float().to("cuda:0")
TensorFakeOuts = torch.full((len(slowa), insy), 0, requires_grad=False).float().to("cuda:0")

TensorOuts = torch.cat((TensorRealOuts, TensorFakeOuts), 0)

print(TensorRealOuts.shape)
print(TensorFakeOuts.shape)

print(TensorOuts.shape)

torch.Size([100, 420])
torch.Size([100, 420])
torch.Size([200, 420])


In [49]:
randoms = 2000

In [50]:
generator = Generator(randoms, insy).to("cuda:0")
classifier = Classifier(insy, insy).to("cuda:0")

In [51]:
generator.apply(weights_init)
classifier.apply(weights_init)

Classifier(
  (fc_in): Linear(in_features=420, out_features=1000, bias=True)
  (fc_h1): Linear(in_features=1000, out_features=1000, bias=True)
  (fc_h2): Linear(in_features=1000, out_features=1000, bias=True)
  (fc_h3): Linear(in_features=1000, out_features=1000, bias=True)
  (fc_out): Linear(in_features=1000, out_features=420, bias=True)
  (activation): ReLU()
  (activation2): Sigmoid()
)

In [52]:
optimizerGenerator = optim.Adam(generator.parameters(), lr=1e-4)
optimizerClassifier = optim.Adam(classifier.parameters(), lr=1e-4)

print(optimizerGenerator)
print(optimizerClassifier)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0001
    weight_decay: 0
)
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0001
    weight_decay: 0
)


In [53]:
criterion = nn.BCELoss()

print(criterion)

BCELoss()


In [54]:
GeneratorLosses = []
ClassifierLosses = []

In [55]:
for epoch in range(30000):
    TensorRealIns = torch.tensor(build_slowa(slowa, make_one_hot_vectors(find_unique(slowa)), insy)).float().to("cuda:0")
    TensorSeeds = torch.rand(len(slowa), randoms, requires_grad=False).float().to("cuda:0")
    TensorFakeIns = generator(TensorSeeds)

    TensorIns = torch.cat((TensorRealIns, TensorFakeIns.detach()), 0)
    
    classifier.train()
    optimizerClassifier.zero_grad()
    
    y_ = classifier(TensorIns)
    lossClassifier = criterion(y_, TensorOuts)
    
    lossClassifier.backward()
    optimizerClassifier.step()
    
    generator.train()
    optimizerGenerator.zero_grad()

    y_ = classifier(TensorFakeIns)
    lossGenerator = criterion(y_, TensorRealOuts)

    lossGenerator.backward()
    optimizerGenerator.step()

    GeneratorLosses.append(lossGenerator)
    ClassifierLosses.append(lossClassifier)
    
#     if lossGenerator < 0.2:
#         break
    
    if epoch % 1000 == 0:
        print(lossClassifier, lossGenerator)
        #print(TensorSeeds)
        #print(TensorFakeIns[0])

tensor(0.6932, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>) tensor(0.6944, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0008, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>) tensor(13.9950, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
tensor(0., device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>) tensor(59.9472, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
tensor(0., device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>) tensor(59.9472, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)


KeyboardInterrupt: 

In [ ]:
fix, ax = plt.subplots()
ax.plot([x for x in range(len(GeneratorLosses))], GeneratorLosses, ".", [x for x in range(len(GeneratorLosses))], ClassifierLosses, ".")